<a href="https://colab.research.google.com/github/pydevcasts/MLHub/blob/master/Handwritten_Digit_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape, y_train.shape)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
(60000, 28, 28) (60000,)


# Preprocess the data

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
num_classes = 10
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# for normalaize pic with 255px
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


# Create the model

In [21]:
batch_size = 128
num_classes = 10
epochs = 10

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,optimizer="adam",metrics=['accuracy'])

# Train the model

In [22]:


# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

# Save the model
model.save('mnist_cnn.h5')
print("Saving the model as mnist.h5")

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.8897 - loss: 0.3536 - val_accuracy: 0.9869 - val_loss: 0.0416
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9793 - loss: 0.0685 - val_accuracy: 0.9898 - val_loss: 0.0292
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9854 - loss: 0.0474 - val_accuracy: 0.9897 - val_loss: 0.0303
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9885 - loss: 0.0391 - val_accuracy: 0.9916 - val_loss: 0.0268
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9911 - loss: 0.0299 - val_accuracy: 0.9914 - val_loss: 0.0255


Saving the model as mnist.h5


# Evaluate the model

In [23]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.025495897978544235
Test accuracy: 0.9914000034332275


# Create GUI to predict digits

In [26]:
# Import necessary libraries
from keras.models import load_model
from google.colab import files
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Load the pre-trained model
model = load_model('mnist.h5')  # Ensure this file is uploaded to Colab

# Function to predict digit
def predict_digit(img):
    img = img.resize((28, 28))  # Resize image to 28x28 pixels
    img = img.convert('L')  # Convert to grayscale
    img = np.array(img)  # Convert to numpy array
    img = img.reshape(1, 28, 28, 1)  # Reshape for the model
    img = img / 255.0  # Normalize pixel values
    res = model.predict(img)  # Predict
    return np.argmax(res), res[0][np.argmax(res)]  # Return predicted digit and confidence

# Upload an image
uploaded = files.upload()

# Process the uploaded image
for fn in uploaded.keys():
    img = Image.open(fn)  # Open the image
    plt.imshow(img, cmap='gray')  # Display the image
    plt.axis('off')  # Hide axis
    plt.show()  # Show image

    digit, acc = predict_digit(img)  # Predict the digit
    print(f'Predicted Digit: {digit}, Confidence: {int(acc * 100)}%')  # Print results